In [1]:
import torch as t
from torch import nn
from torchvision.models import vgg16

In [2]:
def caffevgg16(vgg16path):
    # the 30th layer of features is relu of conv5_3
    model = vgg16(pretrained=False)
    caffevgg = t.load(vgg16path)
    caffekeys = list(caffevgg.keys())
    i = 0
    for name, para in model.state_dict().items():
        para.copy_(caffevgg[caffekeys[i]])
        i += 1
    features = list(model.features)[:30]
    classifier = model.classifier

    classifier = list(classifier)
    del classifier[6]
    use_drop = True
    if not use_drop:
        del classifier[5]
        del classifier[2]
    classifier = nn.Sequential(*classifier)

    # freeze top4 conv
    for layer in features[:10]:
        for p in layer.parameters():
            p.requires_grad = False

    return nn.Sequential(*features), classifier

In [3]:
feature,classifier = caffevgg16('vgg16-00b39a1b.pth')

In [4]:
from torchvision.datasets import VOCDetection
from torch.utils.data import DataLoader
from torchvision import transforms

In [5]:
voc2007 = VOCDetection('./','2007','train',False,transform=transforms.ToTensor())

In [7]:
dataloader = DataLoader(voc2007,1)

In [18]:
for i,(img,xml) in enumerate(dataloader):
    print(xml)
    break

{'annotation': {'folder': ['VOC2007'], 'filename': ['000012.jpg'], 'source': {'database': ['The VOC2007 Database'], 'annotation': ['PASCAL VOC2007'], 'image': ['flickr'], 'flickrid': ['207539885']}, 'owner': {'flickrid': ['KevBow'], 'name': ['?']}, 'size': {'width': ['500'], 'height': ['333'], 'depth': ['3']}, 'segmented': ['0'], 'object': {'name': ['car'], 'pose': ['Rear'], 'truncated': ['0'], 'difficult': ['0'], 'bndbox': {'xmin': ['156'], 'ymin': ['97'], 'xmax': ['351'], 'ymax': ['270']}}}}


In [49]:
from torchvision.datasets import VOCDetection
from torch.utils.data import DataLoader
from torchvision import transforms


class VOCdataloader(DataLoader):
    def __init__(self, vocdevkitpath, year, image_set, batch_size=1, shuffle=False, sampler=None,
                 batch_sampler=None, num_workers=0, collate_fn=t.utils.data._utils.collate.default_collate,
                 pin_memory=False, drop_last=False, timeout=0,
                 worker_init_fn=None):
        self.dataset = VOCDetection(
            vocdevkitpath, year, image_set, False, transform=transforms.ToTensor())
        super().__init__(self.dataset, batch_size=batch_size, shuffle=shuffle, sampler=sampler, num_workers=num_workers,
                         pin_memory=pin_memory, drop_last=drop_last, timeout=timeout, worker_init_fn=worker_init_fn)

In [50]:
vocdataloader = VOCdataloader('./','2007','train')

In [51]:
vocdata = VOCdataloader('./', '2007', 'train')
for i, (image, xml) in enumerate(vocdata):
    print(i)
    break

0
